In [1]:
# Install library yang diperlukan 

import warnings; warnings.simplefilter('ignore')
!pip install unidecode
!pip install textblob
!pip install sastrawi
!pip install tqdm
!pip install wget

In [2]:
# Import library yang diperlukan 
import numpy as np
import pandas as pd
import re
from unidecode import unidecode

from html import unescape
from tqdm import tqdm
import nltk
from nltk import word_tokenize
nltk.download('punkt')
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# modul impor file
import requests
import urllib.request
import wget


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dkd091122\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# URL dataset di GitHub

url = 'https://raw.githubusercontent.com/hafizaah95/DATA/main/subset_dataset.csv'
#url = 'https://raw.githubusercontent.com/hafizaah95/DATA/main/subset_dataset1.csv'
# Membaca dataset langsung dari URL
df = pd.read_csv(url, low_memory=False, encoding='utf8')
df

,Unnamed: 0,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,0,8c03a08a-285c-4e67-a36a-831f579ffbe6,Pu Put,https://play-lh.googleusercontent.com/a/ACg8oc...,aplikasinya bagusss buat belajar bahasa asingg...,5,56,5.139.5,2024-02-21 14:46:17,NaN,NaN,5.139.5
1,1,27635f36-15ed-4c48-9067-c8ce257aa342,Sri Hidayati,https://play-lh.googleusercontent.com/a-/ALV-U...,"asli, Duolingo bagus banget!😭 Dari awal malas ...",5,42,5.137.5,2024-02-05 13:56:41,NaN,NaN,5.137.5
2,2,c03bddcf-ccf4-4307-822d-a55caf84cb47,Stephanie Artha,https://play-lh.googleusercontent.com/a/ACg8oc...,とても good.. aku bisa belajar bahasa Inggris dan...,5,1265,5.139.5,2024-02-17 21:29:06,NaN,NaN,5.139.5
3,3,a86f481c-fa0a-4310-aeef-fb72a71dc344,Linda Saumi (Linda),https://play-lh.googleusercontent.com/a-/ALV-U...,"Aplikasi yang seru, mudah dipahami, ada rank s...",1,453,5.138.5,2024-02-12 20:24:32,NaN,NaN,5.138.5
4,4,4945e054-e64c-4fbb-bc03-6a440e4a4bfe,Silvi Maulidini,https://play-lh.googleusercontent.com/a-/ALV-U...,"aplikasinya seru, kayak game jadi kita gak bos...",5,47,5.137.6,2024-02-15 21:17:20,NaN,NaN,5.137.6
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,07d1763a-be92-4cc0-adb0-39b5666c4068,Karina Putri,https://play-lh.googleusercontent.com/a-/ALV-U...,Menurutku aplikasi ini membantu banget buat be...,5,0,5.13.4,2021-06-24 16:55:35,NaN,NaN,5.13.4
9996,9996,c0bb8b9b-354a-498a-9c9a-cfe898cac570,pudoh pudoh,https://play-lh.googleusercontent.com/a/ACg8oc...,"sangat suka, aplikasi ini sangat membantu buat...",5,0,4.93.7,2021-06-14 11:32:16,NaN,NaN,4.93.7
9997,9997,73d96e5e-d689-4c35-bcc6-a7c79c460ed6,Cllyn Blush,https://play-lh.googleusercontent.com/a-/ALV-U...,Suka banget! Bahasa koreaku di lumayan lancar....,5,0,NaN,2021-05-06 07:57:54,NaN,NaN,NaN
9998,9998,b537bb34-f99f-49b3-9886-d59f9634e226,Maya,https://play-lh.googleusercontent.com/a-/ALV-U...,Sangat terbantu sekali untuk belajar bahasa-ba...,5,0,5.20.3,2021-08-12 17:10:59,NaN,NaN,5.20.3


In [4]:
score_counts = df['score'].value_counts().sort_index()
score_counts

score
1     327
2     203
3     435
4    1140
5    7895
Name: count, dtype: int64

In [5]:
df = df[['content', 'score']]
df

,content,score
0,aplikasinya bagusss buat belajar bahasa asingg...,5
1,"asli, Duolingo bagus banget!😭 Dari awal malas ...",5
2,とても good.. aku bisa belajar bahasa Inggris dan...,5
3,"Aplikasi yang seru, mudah dipahami, ada rank s...",1
4,"aplikasinya seru, kayak game jadi kita gak bos...",5
...,...,...
9995,Menurutku aplikasi ini membantu banget buat be...,5
9996,"sangat suka, aplikasi ini sangat membantu buat...",5
9997,Suka banget! Bahasa koreaku di lumayan lancar....,5
9998,Sangat terbantu sekali untuk belajar bahasa-ba...,5


In [6]:
df[df.duplicated()==True]

,content,score
2780,sangat bagus untuk belajar berbagai bahasa,5
3131,sangat membantu belajar bahasa asing,5
3197,sangat membantu untuk belajar bahasa asing,5
3250,sangat bagus untuk belajar berbagai bahasa,5
3266,sangat membantu dalam belajar bahasa,5
...,...,...
8899,sangat membantu belajar bahasa Inggris,5
8979,sangat membantu belajar bahasa asing,5
8986,sangat bagus buat kita belajar,5
8996,sangat bagus untuk belajar bahasa asing,5


In [7]:
# memeriksa duplikasi
print(df.duplicated().sum())

# membuang duplikasi
df.drop_duplicates(inplace=True)
print(df.shape)
     

74
(9926, 2)


In [8]:
df

,content,score
0,aplikasinya bagusss buat belajar bahasa asingg...,5
1,"asli, Duolingo bagus banget!😭 Dari awal malas ...",5
2,とても good.. aku bisa belajar bahasa Inggris dan...,5
3,"Aplikasi yang seru, mudah dipahami, ada rank s...",1
4,"aplikasinya seru, kayak game jadi kita gak bos...",5
...,...,...
9995,Menurutku aplikasi ini membantu banget buat be...,5
9996,"sangat suka, aplikasi ini sangat membantu buat...",5
9997,Suka banget! Bahasa koreaku di lumayan lancar....,5
9998,Sangat terbantu sekali untuk belajar bahasa-ba...,5


In [10]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
#from gensim.summarization.keypoints import keywords
from gensim.models import TfidfModel, LdaModel

# Preprocess data
factory = StemmerFactory()
stemmer = factory.create_stemmer()

factory_stopword = StopWordRemoverFactory()
stopword = factory_stopword.create_stop_word_remover()

tokenized_data = [[stemmer.stem(word) for word in stopword.remove(doc.lower().split())] for doc in data]

# Create dictionary
dictionary = gensim.corpora.Dictionary(tokenized_data)

# Convert tokenized data to bag-of-words format
bow_corpus = [dictionary.doc2bow(doc) for doc in tokenized_data]

# Create TF-IDF model
tfidf = TfidfModel(bow_corpus)

# Convert data to TF-IDF representation
tfidf_corpus = [tfidf[doc] for doc in bow_corpus]

# Perform topic modeling
lda_model = LdaModel(tfidf_corpus, num_topics=5, id2word=dictionary)

# Get topic distribution for each document
topic_dist = lda_model.get_document_topics(tfidf_corpus)

# Get aspect labels for each topic
aspect_labels = []
for topic in lda_model.print_topics(num_words=4):
    aspect_labels.append(topic.split('"')[1])

# Assign aspect labels to each document
aspects = []
for doc_topic_dist in topic_dist:
    aspect = aspect_labels[np.argmax(doc_topic_dist)]
    aspects.append(aspect)

print(aspects)  # print aspect labels for each document

ModuleNotFoundError: No module named 'gensim'

In [ ]:
import pandas as pd
# Define aspects and rules
aspects = {
    'isi_kursus': ['pelajaran', 'latihan', 'materi', 'kualitas', 'vocabulary', 'bahasa'],
    'antarmuka_pengguna': ['aplikasi', 'desain', 'kegunaan', ' navigasi'],
    'dukungan_pelanggan': ['bantuan', 'dukungan', 'tanggapan', 'masalah']
}

# Create a new column to store the aspect labels
df['aspect'] = ''

# Loop through each review
for index, row in df.iterrows():
    text = row['content']
    for aspect, rules in aspects.items():
        for rule in rules:
            if rule in text:
                df.at[index, 'aspect'] = aspect
                break

# Print the resulting data
print(df)

In [9]:
# modul impor file
import urllib.request
url = 'https://raw.githubusercontent.com/hafizaah95/DATA/main/slang.txt'
##https://raw.githubusercontent.com/taudataanalytics/eLearning/master/data/slang.dic##
##https://raw.githubusercontent.com/taudataanalytics/eLearning/master/data/slang.txt##
response = urllib.request.urlopen(url)
slang_id = response.read().decode('utf-8').split('\n')

print("jumlah slang: ", len(slang_id))

jumlah slang:  2280


In [10]:
import requests

slang_id = {}
response = requests.get('https://raw.githubusercontent.com/hafizaah95/DATA/main/slang.txt')
for line in response.text.splitlines():
    key, value = line.strip().split(':')
    slang_id[key] = value
print("jumlah slang : ", len(slang_id))

jumlah slang :  1962


In [11]:
# modifikasi sesuka hati jika dirasa masih ada slang yang belum tertangani
slang_tambahan = {'geblek': 'goblok', 'smoga': 'semoga', 'trimakasih': 'terima kasih', 'jln2': 'jalan-jalan', 'apk': 'aplikasi', 'bljar': 'belajar',
                  'fitur2': 'fitur', 'moga': 'semoga', 'untk': 'untuk', 'ngk': 'tidak', 'app' : 'aplikasi', 'jdi' : 'jadi'}
slang_id.update(slang_tambahan)

print("jumlah slang : ", len(slang_id))

jumlah slang :  1974


In [12]:

url = 'https://raw.githubusercontent.com/hafizaah95/DATA/main/stopword.txt'
response = urllib.request.urlopen(url)
stopword_id = response.read().decode('utf-8').split('\n')

print("jumlah stopword: ", len(stopword_id))

jumlah stopword:  1349


In [13]:
# modifikasi sesuka hati jika dirasa masih ada stopword yang belum tertangani
stopword_tambahan = set(['a', 'anak', 'allah', 'acara', 'arti', 'asing', 'anak-anak'
                         'ah', 'aja',
                         'b', 'bikin', 'bang', 'bangsa', 'bilang', 'bicara', 'biar', 'bilang', 'belajar', 'bosan',
                         'c', 'cemungut',
                         'd', 'daerah', 'deh', 'dukung', 'duolingo', 'dungolingo', 'download', 'dalam'
                         'e', 'eyes',
                         'f', 'face', 'film', 'folded',
                         'g', 
                         'h', 'hand', 'hitam', 'hands',
                         'i', 'indonesia', 'iya','its',
                         'j', 'jakarta', 'jalan', 'jaring',
                         'k', 'khusus', 'kepala', 'keluarga', 'kah', 'kerja', 'kosakata',
                         'l', 'lihat', 'laki',
                         'm', 'mata', 'mbak', 'malam', 'muda', 'mas',
                         'n', 'name', 'nonton', 'nya', 'nomor',
                         'p', 'provider', 'putih', 'pilih', 'pakai', 'pimpin', 'product', 'program',
                         'q',
                         'r', 'rumah',
                         's', 'smilling', 'smiling', 'si', 'sih', 'suara', 'sinyal', 'sandi', 'salam',
                         't', 'tunggu', 'teman', 'tonton', 'the', 'tuju', 'tears', 'thank', 'terimakasih'
                         'u', 'username', 'url',
                         'v', 
                         'w', 'with',
                         'x',
                         'y', 'yang', 'you'
                         'z',
                         '4g'])
stopword_id = set(stopword_id).union(stopword_tambahan)

print("jumlah stopword: ", len(stopword_id))

jumlah stopword:  1454


In [14]:

def cleaning_content(content):
    # menghapus url
    url_pattern = re.compile(r'(\w+:\/\/\S+)')
    content = url_pattern.sub(' ', content)
    # menghapus hashtag
    hashtag_pattern = re.compile(r'#\w+\b')
    content = hashtag_pattern.sub(' ', content)
    # menghapus nama pengguna (memuat "@")
    username_pattern = re.compile(r'@\w+\b\s*')
    content = username_pattern.sub(' ', content)
    # menghapus angka
    content = re.sub(r'\b\d+\b|[^\w\s]', '', content)
    # mengahpus simbol
    symbol_pattern = re.compile(r'[^\w\s]+')
    content = symbol_pattern.sub(' ', content)
    # menghapus huruf yang berulang
    content = re.sub(r'(\w)(\1{2,})', r'\1', content)
    # Menghapus karakter Jepang
    content = re.sub(r'[\u3040-\u309F\u30A0-\u30FF]+', '', content)
    # menghapus karakter yang tidak diperlukan
    content = re.sub(r'\n', ' ', content)
    content = re.sub(r'\r', ' ', content)
    content = re.sub(r'&\w+', '', content)
    content = re.sub(r'\s+', ' ', content).strip()
    # menangani huruf kapital dan spasi
    content = unidecode(unescape(content.lower()))
    return content

def remove_stopword(content):
    word_tokens = word_tokenize(content)
    filtered_sentence = [w for w in word_tokens if not w in stopword_id]
    return ' '.join(filtered_sentence)

def stemming_and_lemmatization(content):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    return stemmer.stem(content)

def remove_slang(content):
    content = ''.join(slang_id[p] if p in slang_id else p for p in re.split(r'(\W+)', content))
    return content

def remove_repeated_words(content):
    words = content.split()
    seen = set()
    result = []
    for word in words:
        if word not in seen:
            seen.add(word)
            result.append(word)
    return ' '.join(result)

def preprocess(content):
    # cleaning text and lowercase
    output = cleaning_content(content)
    # remove slang
    output = remove_slang(output)
    # stemming and lemmatization
    output = stemming_and_lemmatization(output)
    # remove stopwords
    output = remove_stopword(output)
    #menghapus kata yang berulang
    output = remove_repeated_words(output)
    
    return output

     

In [15]:
df['clean_content'] = ''
for idx, post in tqdm(df.iterrows()):
  df.at[idx, 'clean_content'] = preprocess(post.content)

9926it [1:47:42,  1.54it/s]


In [17]:
df = df[['clean_content', 'score']]
df

,clean_content,score
0,aplikasi bagus buat ajar bahasa asingg dan mud...,5
1,asli bagus banget dari awal malas ajar bahasa ...,5
2,good aku bisa ajar bahasa inggris dan jepang c...,5
3,aplikasi seru mudah paham ada rank tiap minggu...,1
4,aplikasi seru kayak game jadi kita tidak bosen...,5
...,...,...
9995,turut aplikasi ini bantu banget buat ajar baha...,5
9996,sangat suka aplikasi ini bantu buat tingkat ko...,5
9997,suka banget bahasa koreaku di lumayan lancar t...,5
9998,sangat bantu sekali untuk ajar bahasabahasa ba...,5


In [18]:
df.to_csv('cleandata.csv', index = False)

In [ ]:
 # Mendeteksi bahasa
#  try:
        lang = detect(tweet)
        if lang != 'id':
            tweet = ''  # Menghapus teks dalam bahasa selain Bahasa Indonesia
    except:
        pass
    
    return tweet

In [ ]:
def lower_text(content):
    # Mengubah semua huruf menjadi huruf kecil
    content = content.lower()

    return content

# Mengubah teks menjadi huruf kecil pada setiap baris
df['content_lower'] = df['content'].apply(lower_text)

df